## 기본 예시: 프롬프트 + 모델 + 출력 파서

가장 기본적이고 일반적인 사용 사례는 prompt 템플릿과 모델을 함께 연결하는 것입니다. 이것이 어떻게 작동하는지 보기 위해, 각 나라별 수도를 물어보는 Chain을 생성해 보겠습니다.


In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

### PromptTemplate


In [2]:
from langchain.prompts import PromptTemplate

`from_template()` 메소드를 사용하여 PromptTemplate 객체 생성


In [3]:
# template 정의
template = "{country}의 수도는 어디인가요?"

# from_template 메소드를 이용하여 PromptTemplate 객체 생성
prompt_template = PromptTemplate.from_template(template)
prompt_template

PromptTemplate(input_variables=['country'], template='{country}의 수도는 어디인가요?')

In [4]:
# prompt 생성
prompt = prompt_template.format(country="대한민국")
prompt

'대한민국의 수도는 어디인가요?'

In [5]:
# prompt 생성
prompt = prompt_template.format(country="미국")
prompt

'미국의 수도는 어디인가요?'

### Model(LLM)

`ChatOpenAI`

OpenAI 사의 채팅 전용 Large Language Model(llm) 입니다.

객체를 생성할 때 다음을 옵션 값을 지정할 수 있습니다. 옵션에 대한 상세 설명은 다음과 같습니다.

`temperature`

- 사용할 샘플링 온도는 0과 2 사이에서 선택합니다. 0.8과 같은 높은 값은 출력을 더 무작위하게 만들고, 0.2와 같은 낮은 값은 출력을 더 집중되고 결정론적으로 만듭니다.

`max_tokens`

- 채팅 완성에서 생성할 토큰의 최대 개수입니다.

`model_name`: 적용 가능한 모델 리스트

- 링크: https://www.teddynote.com/openai/openai-models/
- 링크: https://platform.openai.com/docs/models


In [6]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-3.5-turbo",
    max_tokens=2048,
    temperature=0.1,
)

### Output


In [7]:
from langchain_core.output_parsers import StrOutputParser

output = StrOutputParser()

## Chain 생성

**LCEL(LangChain Expression Language)**

여기서 우리는 LCEL을 사용하여 다양한 구성 요소를 단일 체인으로 결합합니다

```
chain = prompt | model | output_parser
```

`|` 기호는 [unix 파이프 연산자](<https://en.wikipedia.org/wiki/Pipeline_(Unix)>)와 유사하며, 서로 다른 구성 요소를 연결하고 한 구성 요소의 출력을 다음 구성 요소의 입력으로 전달합니다.

이 체인에서 사용자 입력은 프롬프트 템플릿으로 전달되고, 그런 다음 프롬프트 템플릿 출력은 모델로 전달되며, 그 다음 모델 출력은 출력 파서로 전달됩니다. 각 구성 요소를 개별적으로 살펴보면 무슨 일이 일어나고 있는지 이해할 수 있습니다.


In [8]:
# 주어진 나라에 대하여 수도를 묻는 프롬프트 템플릿을 생성합니다.
# prompt = PromptTemplate.from_template("{country}의 수도는 어디인가요?")

# 주어진 나라에 대하여 수도를 묻는 프롬프트 템플릿을 생성합니다.
template = """
당신은 친절하게 답변해 주는 친절 봇입니다. 사용자의 질문에 [FORMAT]에 맞추어 답변해 주세요.
답변은 항상 한글로 작성해 주세요.

질문:
{question}에 대하여 설명해 주세요.

FORMAT:
- 개요:
- 예시:
- 출처:
"""

template = """
당신은 영어를 가르치는 10년차 영어 선생님입니다. 상황에 [FORMAT]에 영어 회화를 작성해 주세요.

상황:
{question}

FORMAT:
- 영어 회화:
- 한글 해석:
"""


prompt = PromptTemplate.from_template(template)


# OpenAI 챗모델을 초기화합니다.
model = ChatOpenAI(model="gpt-4-turbo-preview")
# 문자열 출력 파서를 초기화합니다.
output_parser = StrOutputParser()

# 프롬프트, 모델, 출력 파서를 연결하여 처리 체인을 구성합니다.
chain = prompt | model | output_parser

# 완성된 Chain 을 이용하여 country 를 '대한민국'으로 설정하여 실행합니다.
# chain.invoke({"country": "대한민국"})
print(chain.invoke({"question": "저는 식당에 가서 음식을 주문하고 싶어요"}))

영어 회화:
- Excuse me, I'd like to order some food, please.
- Sure, what would you like to have?
- Could I get the grilled salmon with a side of vegetables, please?
- Absolutely, would you like anything to drink with that?
- Yes, I'll have a glass of white wine, please.
- Great choice. I'll have your order right up. Thank you.

한글 해석:
- 실례합니다, 음식을 주문하고 싶어요.
- 네, 무엇을 드시겠어요?
- 구운 연어와 야채 반찬 하나 주세요.
- 알겠습니다, 음료는 무엇으로 드릴까요?
- 네, 화이트 와인 한 잔 주세요.
- 좋은 선택이네요. 주문 바로 준비하겠습니다. 감사합니다.


In [11]:
# 완성된 Chain 을 이용하여 question 를 '미국에서 피자 주문'으로 설정하여 실행합니다.
print(chain.invoke({"question": "미국에서 피자 주문"}))

영어 회화:
- Hello, I'd like to order a pizza for delivery, please.
- Sure, what would you like?
- I’d like a large pepperoni pizza with extra cheese.
- Anything else?
- Yes, can I also have a 2-liter bottle of Coke?
- Of course. Can I have your address, please?
- It’s 123 Maple Street, Apartment 4B.
- Your total comes to $22.50. It should be there in about 30-40 minutes.
- Great, thank you!

한글 해석:
- 안녕하세요, 배달로 피자 주문하고 싶습니다.
- 네, 무엇을 드릴까요?
- 큰 사이즈의 페퍼로니 피자에 치즈를 추가로 올려주세요.
- 더 필요한 것이 있으십니까?
- 네, 2리터짜리 콜라도 하나 주문할 수 있을까요?
- 물론입니다. 주소를 알려주시겠어요?
- 123 메이플 스트리트, 아파트 4B입니다.
- 총 금액은 $22.50이며, 대략 30-40분 내에 배달될 예정입니다.
- 좋습니다, 감사합니다!


## 전체 파이프라인

단계를 따라가려면:

1. 원하는 주제에 대한 사용자 입력을 `{"topic": "양자역학"}`로 전달합니다.
2. `prompt` 컴포넌트는 사용자 입력을 받아 `topic`을 사용하여 프롬프트를 구성한 후 PromptValue를 생성합니다.
3. `model` 컴포넌트는 생성된 프롬프트를 가져와 OpenAI LLM 모델에 평가를 위해 전달합니다. 모델에서 생성된 출력은 `ChatMessage` 객체입니다.
4. 마지막으로, `output_parser` 컴포넌트는 `ChatMessage`를 받아 이를 Python 문자열로 변환하며, 이는 invoke 메서드에서 반환됩니다.

`prompt` 또는 `prompt | model`과 같은 체인의 작은 버전을 테스트하여 중간 결과를 확인하고자 하는 경우에 대해 유의하세요.

`prompt` 객체와 `model` 객체를 사용하여 대화형 AI 시스템을 시뮬레이션합니다. 먼저, `prompt.invoke` 메서드를 호출하여 사용자 입력에 대한 프롬프트 값을 생성합니다. 그런 다음, `prompt` 객체와 `model` 객체를 파이프 연산자(`|`)로 연결하고 `invoke` 메서드를 호출하여 AI 모델이 생성한 응답을 반환합니다.

이 과정에서 AI 모델은 '양자역학'이라는 주제에 대한 쉽게 이해할 수 있는 답변을 생성합니다.


![](./images/lcel-flow.png)


In [12]:
# prompt 를 PromptTemplate 객체로 생성합니다.
prompt = PromptTemplate.from_template("{topic} 에 대해 쉽게 설명해주세요.")

# input 딕셔너리에 주제를 'ice cream'으로 설정합니다.
input = {"topic": "양자역학"}

# prompt 객체의 invoke 메서드를 사용하여 input을 전달하고 대화형 프롬프트 값을 생성합니다.
prompt.invoke(input)

# prompt 객체와 model 객체를 파이프(|) 연산자로 연결하고 invoke 메서드를 사용하여 input을 전달합니다.
# 이를 통해 AI 모델이 생성한 메시지를 반환합니다.
(prompt | model).invoke(input)

AIMessage(content="양자역학은 물질과 에너지의 가장 기본적인 단위인 양자의 행동과 상호작용을 설명하는 물리학의 한 분야입니다. 우리가 일반적으로 경험하는 매크로 세계와는 달리, 양자 세계에서는 직관적으로 이해하기 어려운 여러 현상이 나타납니다. 이를 쉽게 설명하기 위해 몇 가지 핵심 개념을 소개하겠습니다.\n\n1. **양자의 개념**: 양자는 에너지나 물질의 가장 작은 단위로, 이산적(끊어져 있는) 값을 가집니다. 예를 들어, 전자가 원자핵 주위를 도는 에너지 레벨은 연속적이지 않고 특정 양자 상태에만 위치할 수 있습니다.\n\n2. **파동-입자 이중성**: 양자역학에서는 물질과 빛이 동시에 파동과 입자의 성질을 가진다고 설명합니다. 예를 들어, 빛은 파동처럼 간섭과 회절 현상을 보이면서도, 입자처럼 에너지를 이산적인 양인 광자의 형태로 전달할 수 있습니다.\n\n3. **불확정성 원리**: 하이젠베르크의 불확정성 원리에 따르면, 입자의 위치와 운동량(속도와 질량의 곱)을 동시에 정확하게 알 수 없습니다. 한 쪽을 더 정확하게 알게 되면, 다른 쪽은 더 불확실해집니다. 이는 양자 세계의 기본적인 속성 중 하나입니다.\n\n4. **양자 중첩**: 양자는 여러 가능한 상태의 '중첩' 상태에 있을 수 있으며, 이는 관측되기 전까지는 여러 상태가 '동시에' 존재할 수 있음을 의미합니다. 관측하는 순간, 양자는 특정한 상태로 '붕괴'됩니다.\n\n5. **양자 얽힘**: 두 개의 입자가 얽힌 상태에 있으면, 하나의 입자의 상태를 측정하면 즉시 다른 입자의 상태도 결정됩니다. 이 현상은 입자들이 얼마나 멀리 떨어져 있든 상관없이 즉각적으로 일어나며, '스푸키 액션 앳 어 디스턴스(유령 같은 원거리 작용)'라고도 불립니다.\n\n양자역학은 현대 과학과 기술에 깊은 영향을 미쳤습니다. 예를 들어, 반도체, 레이저, 양자 컴퓨팅 등이 이 원리를 기반으로 합니다. 양자역학의 개념은 직관에 반하는 경우가 많지만, 실험을 통해 그 예측이 정확함이 반복적으로 증명

In [13]:
# parse_output 메서드를 사용하여 AI 모델이 생성한 메시지 문자열로 출력합니다.
(prompt | model | output_parser).invoke(input)

"양자역학은 물리학의 한 분야로, 아주 작은 입자들의 세계에서 일어나는 현상을 설명하는 이론입니다. 일상생활에서는 경험하기 어려운, 원자나 전자와 같은 미소한 입자들의 행동을 다룹니다. 양자역학을 이해하기 위해서는 몇 가지 중요한 개념을 알아야 합니다.\n\n1. **양자**: '양자'란 '양'을 의미하는데, 여기서는 에너지가 불연속적인 최소 단위로 존재한다는 개념입니다. 즉, 에너지가 연속적인 흐름이 아니라 '양자'라고 불리는 일정한 '덩어리'로만 전달되거나 흡수된다는 뜻입니다.\n\n2. **파동-입자 이중성**: 양자역학에서는 빛과 같은 입자들이 때로는 파동처럼, 때로는 입자처럼 행동한다는 것을 설명합니다. 즉, 같은 것이 상황에 따라 파동이 될 수도 있고 입자가 될 수도 있다는 것입니다.\n\n3. **불확정성 원리**: 하이젠베르크의 불확정성 원리에 따르면, 우리는 입자의 위치와 운동량(속도와 방향)을 동시에 정확하게 알 수 없습니다. 한 쪽을 더 정확하게 알려고 하면 다른 한 쪽은 더 불확실해집니다. 이는 양자 세계에서의 불확실성이 근본적인 성질임을 의미합니다.\n\n4. **양자 중첩**: 입자가 두 가지 이상의 상태에 동시에 존재할 수 있다는 개념입니다. 예를 들어, 전자가 동시에 여러 위치에 존재할 수 있습니다. 이 상태는 관측될 때까지 유지되며, 관측하는 순간 하나의 상태로 '붕괴'됩니다.\n\n5. **양자 얽힘**: 두 개 이상의 입자가 서로 얽혀서 한 입자의 상태를 결정하면 즉시 다른 입자의 상태도 결정되는 현상입니다. 이는 입자들이 서로 멀리 떨어져 있어도 적용되며, 양자역학의 가장 신비로운 부분 중 하나입니다.\n\n양자역학은 이와 같은 비직관적이고 독특한 원리들로 인해 초기에는 많은 논란을 일으켰습니다. 하지만 현재는 물리학의 근간을 이루는 중요한 이론으로, 컴퓨터, 통신, 의료 등 다양한 분야에서 응용되고 있습니다."